In [1]:
import os
from dotenv import load_dotenv
from langfuse import Langfuse
from langfuse.openai import AzureOpenAI
 
load_dotenv()
 
langfuse = Langfuse()  # Auto-reads LANGFUSE_* env vars
 
AZURE_CLIENT = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-08-01-preview",
)
JUDGE_MODEL = os.getenv("AZURE_OPENAI_MODEL")

In [171]:
import pprint

In [34]:
x = langfuse.api.observations.get_many(trace_id="e64a391779b5b9fcb052c0bce5ec2377")
# x = langfuse.api.trace.get(trace_id="71c11fa1370bed39e08218f48943afa2")
trace=x.dict()
trace

{'data': [{'id': 'd273d62612c0bcf4',
   'traceId': 'e64a391779b5b9fcb052c0bce5ec2377',
   'type': 'GENERATION',
   'name': 'call_llm',
   'startTime': datetime.datetime(2025, 11, 26, 15, 50, 7, 340000, tzinfo=datetime.timezone.utc),
   'endTime': datetime.datetime(2025, 11, 26, 15, 50, 10, 960000, tzinfo=datetime.timezone.utc),
   'completionStartTime': None,
   'model': 'azure/gpt-4o-mini',
   'modelParameters': {},
   'input': {'model': 'azure/gpt-4o-mini',
    'config': {'system_instruction': '\n        **Role:** Host agent for the agent-to-agent protocol; delegates queries to specialized remote agents with maximum efficiency.\n\n**Core Directives:**\n\n* **Task Delegation:** Use the `send_message` function to assign precise, actionable tasks to remote agents.\n* **Full Context Provision:** If an agent repeatedly asks for user confirmation, it likely lacks conversation history. Include all relevant context in the task to prevent this.\n* **Autonomous Multi-Agent Engagement:** Engage

In [35]:
output= None
for obs in trace['data']:
                if obs['type'] == "GENERATION":
                        input = obs['input']['contents'][0]['parts'][0]['text']
                        contents = obs['output']['content']
                        for sub_content in contents['parts']:
                                if sub_content.get('text'):
                                        output = sub_content.get('text')
                        trace_id = obs['traceId']
                        startTime = obs['startTime']
                        model = obs['model']
                        level = trace['meta']['totalItems']
                        environment = obs['environment']
                        input_tokens = obs['usageDetails']['input']
                        output_tokens = obs['usageDetails']['output']
                        total_tokens = obs['usageDetails']['total']
                        latency = obs['latency']
                    

                if output== None:
                    Agents = [obs for obs in trace['data'] if obs.get('type') == 'AGENT']
                    latest_agent = min(Agents, key=lambda x: x['endTime'])
                    # output = latest_agent['output']['result']['artifacts'][0]['parts']['text']
                    artifacts = latest_agent['output']['result']['artifacts']
                    first_artifact = artifacts[0]
                    parts = first_artifact['parts']
                    output = parts[0]['text']

In [36]:
print(input,output,trace_id,startTime,model,level,environment,input_tokens,output_tokens,total_tokens,latency)

summarize agentic framework for me  ### Das dritte Newtonsche Gesetz (Actio = Reactio)

#### Grundprinzip
**"Zu jeder Aktion gibt es eine gleich große, aber entgegengerichtete Reaktion."**  
Wenn ein Körper A auf einen Körper B eine Kraft ausübt, wirkt Körper B gleichzeitig mit einer gleich großen, aber in entgegengesetzter Richtung wirkenden Kraft auf Körper A zurück.

#### Mathematische Formulierung
**F₁₂ = -F₂₁**  
- F₁₂: Kraft, die Körper 1 auf Körper 2 ausübt  
- F₂₁: Kraft, die Körper 2 auf Körper 1 ausübt  
- Das Minuszeichen zeigt die entgegengesetzte Richtung an.

#### Wichtige Eigenschaften
- **Gleichzeitigkeit**: Aktions- und Reaktionskraft treten immer gleichzeitig auf, nie nacheinander.
- **Gleiche Stärke**: Beide Kräfte haben exakt den gleichen Betrag.
- **Entgegengesetzte Richtung**: Die Kräfte wirken in genau entgegengesetzten Richtungen.
- **Verschiedene Angriffspunkte**: Die Kräfte greifen an unterschiedlichen Körpern an.

#### Alltägliche Beispiele
- **Gehen**: Fuß d

In [37]:
output = None
for obs in trace['data']:
        if obs['type'] == "GENERATION":
                input = obs['input']['contents'][0]['parts'][0]['text']
                contents = obs['output']['content']
                for sub_content in contents['parts']:
                        if sub_content.get('text'):
                                output = sub_content.get('text')
                trace_id = obs['traceId']
                startTime = obs['startTime']
                model = obs['model']
                level = trace['meta']['totalItems']
                environment = obs['environment']
                input_tokens = obs['usageDetails']['input']
                output_tokens = obs['usageDetails']['output']
                total_tokens = obs['usageDetails']['total']
                latency = obs['latency']

        if output== None:
                Agents = [obs for obs in trace['data'] if obs.get('type') == 'AGENT']
                latest_agent = min(Agents, key=lambda x: x['endTime'])
                # output = latest_agent['output']['result']['artifacts'][0]['parts']['text']
                artifacts = latest_agent['output']['result']['artifacts']
                first_artifact = artifacts[0]
                parts = first_artifact['parts']
                output = parts[0]['text']


In [38]:
print(trace_id,startTime,model,level,environment,input_tokens,output_tokens,total_tokens,latency)

e64a391779b5b9fcb052c0bce5ec2377 2025-11-26 15:49:55.428000+00:00 azure/gpt-4o-mini 10 default 3670 51 3721 11910.0


In [14]:
print(input)
print(output)

tell me the capital of pakistan then tell me 10 facts about that capital
The capital of Pakistan is **Islamabad**.

Here are 10 interesting facts about Islamabad:

1. **Purpose-Built Capital**  
Islamabad was specifically designed and constructed in the 1960s to replace Karachi as Pakistan's capital. The city was planned from scratch by Greek architect Constantinos Apostolou Doxiadis.

2. **Geometric Grid Design**  
The city follows a unique grid pattern with sectors numbered and lettered systematically. Each sector is designated by a number and letter combination (like F-6, G-7), making navigation remarkably organized.

3. **Green City by Design**  
Islamabad is one of the world's greenest capitals, with over 70% of the city covered by natural vegetation and parks. It was specifically planned to maintain extensive green spaces.

4. **Twin City Connection**  
Islamabad forms a twin city with neighboring Rawalpindi. While Islamabad serves as the administrative center, Rawalpindi provide

In [10]:
traces = []
for obs in trace['data']:
    if obs['type'] == "AGENT":
        traces.append(obs)

In [166]:
latest_agent = min(traces, key=lambda x: x['endTime'])
pprint.pprint(latest_agent)

{'calculatedInputCost': None,
 'calculatedOutputCost': None,
 'calculatedTotalCost': 0.0,
 'completionStartTime': None,
 'completionTokens': 0,
 'costDetails': {},
 'createdAt': '2025-11-22T14:44:56.533Z',
 'endTime': datetime.datetime(2025, 11, 22, 14, 44, 55, 7000, tzinfo=datetime.timezone.utc),
 'environment': 'default',
 'id': 'dd9cc781aa7ab644',
 'input': {'args': [],
           'kwargs': {'message_request': {'id': '27a1ec3f-3e7c-4410-8318-503639adc462',
                                          'jsonrpc': '2.0',
                                          'method': 'message/send',
                                          'params': {'configuration': None,
                                                     'message': {'contextId': '52bf3458-b345-430d-82cd-136848af21c6',
                                                                 'extensions': None,
                                                                 'kind': 'message',
                                             

In [168]:
output = latest_agent['output']['result']['artifacts'][0]['parts']['text']


TypeError: list indices must be integers or slices, not str

In [167]:
# Sort the agent observations by endTime (ascending), then take the last one

artifacts = latest_agent['output']['result']['artifacts']
first_artifact = artifacts[0]                    # usually only one
parts = first_artifact['parts']
output_text = parts[0]['text']
pprint.pprint(output)

'The manager of the Marketing department is Alejandro Brender.'


In [154]:
pprint.pprint(traces)

[{'calculatedInputCost': None,
  'calculatedOutputCost': None,
  'calculatedTotalCost': 0.0,
  'completionStartTime': None,
  'completionTokens': 0,
  'costDetails': {},
  'createdAt': '2025-11-22T14:45:06.873Z',
  'endTime': datetime.datetime(2025, 11, 22, 14, 45, 5, 728000, tzinfo=datetime.timezone.utc),
  'environment': 'default',
  'id': 'c8dc3cabb5c8a4d1',
  'input': {'args': [],
            'kwargs': {'message_request': {'id': '632f598c-172c-4a11-8dc2-33946b223ad2',
                                           'jsonrpc': '2.0',
                                           'method': 'message/send',
                                           'params': {'configuration': None,
                                                      'message': {'contextId': '082ade9b-61eb-4d66-92ca-1d7ef80df020',
                                                                  'extensions': None,
                                                                  'kind': 'message',
                          

In [112]:
INPUT_COST = 0.15 / 1000000
OUTPUT_COST = 0.60 / 1000000

def calculate_cost(input_tokens, output_tokens):
    input_cost = input_tokens * INPUT_COST
    output_cost = output_tokens * OUTPUT_COST
    total_cost = input_cost + output_cost

    return input_cost, output_cost, total_cost

In [113]:
cost = calculate_cost(input_tokens, output_tokens)

In [119]:
print(cost[1])

7.8e-06


In [128]:
pprint.pprint(id)
pprint.pprint(input)
pprint.pprint(startTime)
pprint.pprint(model)
pprint.pprint(level)
pprint.pprint(environment)
pprint.pprint(input_tokens)
pprint.pprint(output_tokens)
pprint.pprint(total_tokens)

'6f338e14efc427b291ecb6fce691bfae'
'what is the capital of Pakistan and write 10 lines about it.'
datetime.datetime(2025, 11, 20, 9, 28, 20, 473000, tzinfo=datetime.timezone.utc)
'azure/gpt-4o-mini'
7
'default'
921
38
959


In [39]:
import json
import os
from datetime import  datetime
from typing import Any, Dict
import pandas as pd
from dotenv import load_dotenv
from langfuse import Langfuse
from langfuse.openai import AzureOpenAI

load_dotenv()

langfuse = Langfuse()
AZURE_CLIENT = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-08-01-preview",
)
JUDGE_MODEL = os.getenv("AZURE_OPENAI_MODEL")
class AgentEvaluator:
    def __init__(self):
        pass

    def evaluate_agent_trace(trace_id,trace,input,output,steps:list) -> Dict[str, Any]:
        trace = trace
        question = input
        final_answer = output
        full_conversation = steps

        scores: Dict[str, Any] = {}
        

        success_rate = AgentEvaluator.llm_judge(
            prompt_template="""
                Question: {question}
                Final Answer: {answer}

    Did the agent fully solve the user's original goal or question?
    Score 0.0–1.0 (1.0 = completely solved, no missing parts)
    Respond ONLY with valid JSON: {{"score": 0.XX, "explanation": "explanation of the score "}}
            """,
            question=question,
            answer=final_answer,
        )

    #     tool_selection_accuracy = AgentEvaluator.llm_judge(
    #         prompt_template="""
    # Question: {question}
    # All tool calls made: {tools_used}

    # Did the agent select the correct tools and in a logical order?
    # Score 0.0–1.0
    # JSON only: {{"score": 0.XX, "explanation": "..."}}
    #         """,
    #         question=question,
    #         tools_used=total_calls,
    #         # tools_used=llm_call+tool_call
    #     )

        reasoning_quality = AgentEvaluator.llm_judge(
            prompt_template="""
                Full conversation (including thoughts and tool results):
                {conversation}

                Are the intermediate reasoning steps logical, coherent, and free of major jumps?
                Score 0.0–1.0
                JSON only.
                        """,
            conversation=full_conversation,
        )
        judge_answer_groundedness = AgentEvaluator.llm_judge(
                prompt_template = """
            You are evaluating whether the final answer is 100% grounded in real tool results or verifiable facts from the conversation.

            Question: {input}

            Full agent conversation (thoughts + tool calls + tool results):
            {steps}

            Final Answer given to user:
            {output}

            Rules:
            - Score 1.0 only if EVERY factual claim in the Final Answer can be directly traced to a real tool result or observation in the conversation.
            - Score 0.9–0.99 if almost everything is grounded but one tiny non-critical detail is missing or slightly rephrased.
            - Score 0.8 or lower if any important claim is invented, assumed, or not backed by tools.
            - If the agent says "I couldn't find...", "No results", or admits uncertainty → that IS grounded (as long as it's true).

            Respond ONLY with valid JSON:
            {{
            "score": 0.XX,
            "explanation": "Brief justification listing any ungrounded claims (or 'Fully grounded' if score >= 0.95)"
            }}
            """,
            input=question,
            steps=full_conversation,
            output=final_answer,
        )



        hallucinated_tool_output = AgentEvaluator.llm_judge(
            prompt_template="""
                Question: {question}
                Conversation with tool results: {conversation}

                Did the agent ever invent or hallucinate a tool result that wasn't actually returned?
                0.0 = yes (bad), 1.0 = no hallucinations
                JSON only.
                        """,
            question=question,
            conversation=full_conversation,
        )

        conciseness = AgentEvaluator.llm_judge(
            prompt_template="""
                Final Answer: {answer}

                Is the final response concise and to-the-point (no unnecessary fluff/repetition)?
                Score 0.0–1.0
                JSON only.
                        """,
            answer=final_answer,
        )

        start = getattr(trace, "start_time", None) or getattr(trace, "timestamp", None)
        end = getattr(trace, "end_time", None) or datetime.now()
        latency = (end - start).total_seconds() if (start and end) else None

        scores["total_steps"] = steps
        scores["latency_seconds"] = round(latency, 2) if latency is not None else None
        usage = getattr(trace, "usage", None)
        scores["total_tokens"] = (
            usage.total_tokens
            if usage and getattr(usage, "total_tokens", None) is not None
            else getattr(trace, "total_tokens", 0)
        )
        task_success_binary = 1.0 if success_rate >= 0.90 else 0.0


        # plan_adherence = AgentEvaluator.llm_judge(
        #     prompt_template="""
        #         Full trace (thoughts + actions): {conversation}

        #         First, extract the agent's explicit or implicit plan from the reasoning steps.
        #         Then evaluate: Did the agent follow its own plan without major unnecessary deviations?
        #         Score 0.0–1.0 (1.0 = perfectly adhered to a reasonable plan)
        #         Respond ONLY with valid JSON: {{"score": 0.XX, "explanation": "short reasoning about adherence"}}
        #             """,
        #     conversation=full_conversation,
        # )

        plan_quality = AgentEvaluator.llm_judge(
            prompt_template="""
                Task/Question: {question}
                Extracted plan from agent's thoughts: {conversation}

                Evaluate the quality of the plan itself (regardless of execution): Is it logical, efficient, comprehensive, and likely to solve the task with minimal steps?
                Score 0.0–1.0 (1.0 = optimal or near-optimal plan)
                Respond ONLY with valid JSON: {{"score": 0.XX, "explanation": "short reasoning about plan quality"}}
                    """,
            question=question,
            conversation=full_conversation,
        )
        judge_reasoning_traceability = AgentEvaluator.llm_judge(
            prompt_template="""
        Examine the agent's full reasoning trace (thoughts, actions, observations).

        Full trace:
        {full_conversation}

        {question}
        Evaluate reasoning traceability / human readability:

        - Are the agent's thoughts clearly written in natural language?
        - Does each thought explain WHY the next action is being taken?
        - Are there confusing jumps, missing explanations, or vague thoughts like "I think..." without justification?
        - Is it easy for a competent human to follow the exact logic flow from start to finish?

        Scoring:
        - 1.0 = Crystal clear, every step has a explicit "because..." or strong implicit logical link
        - 0.9 = Very good, only 1–2 minor jumps
        - 0.7–0.8 = Mostly followable but several unclear leaps
        - ≤0.6 = Frequently confusing or thoughts are missing/cryptic

        Respond ONLY with valid JSON:
        {{
        "score": 0.XX,
        "explanation": "Short summary of clarity and any confusing parts"
        }}
        """,
            question=question,
            full_conversation=full_conversation,
        )


        success_rate= success_rate,
        # tool_selection_accuracy= tool_selection_accuracy,
        reasoning_quality= reasoning_quality,
        judge_answer_groundedness= judge_answer_groundedness,
        hallucinated_tool_output= hallucinated_tool_output,
        conciseness= conciseness,
        task_success_binary= task_success_binary,
        # plan_adherence= plan_adherence,
        plan_quality= plan_quality,
        judge_reasoning_traceability= judge_reasoning_traceability,
    
        return success_rate,reasoning_quality,judge_answer_groundedness,hallucinated_tool_output,conciseness,task_success_binary,plan_quality,judge_reasoning_traceability


    def llm_judge(prompt_template: str, **kwargs) -> float:

        full_prompt = prompt_template.strip().format(**kwargs)

        try:
            response = AZURE_CLIENT.chat.completions.create(
                model=JUDGE_MODEL,
                temperature=0.0,
                response_format={"type": "json_object"},
                max_tokens=500,
                messages=[{"role": "system", "content": full_prompt}],
            )
            content = response.choices[0].message.content.strip()
            result = json.loads(content)
            return float(result.get("score", 0.0))
        except Exception as e:
            raw_content = locals().get("content", "N/A")
            print(f"Judge failed: {e}\nRaw: {raw_content}")
            return 0.0
